In [1]:
using CompTime, InteractiveUtils

In [2]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

In [3]:
function pw_rec(n, x)
    if iszero(n)
        one(x)
    else
        x * pw_rec(n - 1, x)
    end
end

pw_rec (generic function with 1 method)

In [4]:
Tuple(pw_rec(i, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [5]:
function pw_loop(n, x)
    r = one(x)
    for i in 1:n
        r = r * x
    end
    return r
end

pw_loop (generic function with 1 method)

In [6]:
Tuple(pw_loop(i, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [7]:
@ct_enable function pw_ct(@ct(n), x::T) where {T}
    r = @ct(one(T))
    @ct_ctrl for i in 1:n
        r = r * x
    end
    return r
end

runtime (generic function with 1 method)

In [8]:
pw_ct(n, x) = pw_ct(Val{n}, x)

pw_ct (generic function with 2 methods)

In [9]:
Tuple(pw_ct(i, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [10]:
Tuple(runtime(pw_ct, i, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [11]:
Tuple(comptime(pw_ct, Val{i}, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [12]:
@code_typed pw_ct(Val{3}, 10)

CodeInfo(
1 ─ %1 = Base.mul_int(1, x)::Int64
│   %2 = Base.mul_int(%1, x)::Int64
│   %3 = Base.mul_int(%2, x)::Int64
└──      return %3
) => Int64

In [13]:
@code_lowered runtime(pw_ct, 3, 10)

CodeInfo(
1 ─ %1  = $(Expr(:static_parameter, 1))
│         r = Main.one(%1)
│   %3  = 1:n
│         @_5 = Base.iterate(%3)
│   %5  = @_5
│   %6  = %5 === nothing
│   %7  = Base.not_int(%6)
└──       goto #4 if not %7
2 ┄ %9  = @_5
│         i = Core.getfield(%9, 1)
│   %11 = Core.getfield(%9, 2)
│   %12 = r
│         r = %12 * x
│         @_5 = Base.iterate(%3, %11)
│   %15 = @_5
│   %16 = %15 === nothing
│   %17 = Base.not_int(%16)
└──       goto #4 if not %17
3 ─       goto #2
4 ┄ %20 = r
└──       return %20
)

In [14]:
debug(pw_ct, Val{3}, 10) |> cleanup

quote
    r = 1
    r = r * x
    r = r * x
    r = r * x
    return r
end

In [15]:
function pw_pe(::Val{n}, x) where {n}
    r = one(x)
    for i in 1:n
        r = r * x
    end
    return r
end

pw_pe (generic function with 1 method)

In [16]:
Tuple(pw_pe(Val(i), "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [17]:
@code_typed pw_pe(Val(3), 10)

CodeInfo(
1 ─       nothing::Nothing
2 ┄ %2  = φ (#1 => 1, #6 => %10)::Int64
│   %3  = φ (#1 => 1, #6 => %4)::Int64
│   %4  = Base.mul_int(%3, x)::Int64
│   %5  = (%2 === 3)::Bool
└──       goto #4 if not %5
3 ─       goto #5
4 ─ %8  = Base.add_int(%2, 1)::Int64
└──       goto #5
5 ┄ %10 = φ (#4 => %8)::Int64
│   %11 = φ (#3 => true, #4 => false)::Bool
│   %12 = Base.not_int(%11)::Bool
└──       goto #7 if not %12
6 ─       goto #2
7 ─       return %4
) => Int64